In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
#import randomforest from sklearn 
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from xgboost import XGBClassifier

In [17]:
# importing the dataset from the csv file in /data 
df = pd.read_csv('./data/train.csv')
df.head()

dftest = pd.read_csv('./data/test.csv')
dftest.head()

,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,id
0,66149,0.352453,0.079074,0.049762,0.000113,2182.571837,1.068958e+06,2793.933194,365598.217794,4882.736629,...,54.443741,-7.310032,54.092850,-3.083705,21.973936,-6.226634,47.559479,-10.531331,40.015507,9586
1,66149,0.478756,0.087817,0.161004,0.006013,2377.436870,4.982766e+05,2238.413578,145786.540526,4952.139799,...,68.952118,-5.488387,47.951576,4.220189,54.550598,-1.984476,83.323364,1.572527,59.916058,6984
2,66149,0.329632,0.076669,0.191606,0.000796,2525.119913,1.121766e+05,2472.943874,152231.535952,5245.819561,...,34.014881,-2.187970,48.011997,-5.302452,20.189150,-1.775238,33.344528,-1.861419,39.216839,7047
3,66149,0.302360,0.092979,0.106443,0.001762,1609.587478,1.552065e+05,2010.500266,66792.319500,3505.274189,...,61.442478,-7.453752,63.574467,6.466436,26.298433,-4.502402,47.922512,-4.084498,32.702728,2510
4,66149,0.400731,0.090858,0.125889,0.006567,1235.357695,2.267020e+05,1739.174077,218522.407816,2733.308857,...,48.944210,-2.122545,53.329845,-2.181120,41.637207,-3.361829,46.104019,-2.228968,30.943306,34


In [18]:
genres_list = [
    'blues',
    'classical',
    'country',
    'disco',
    'hiphop',
    'jazz',
    'metal',
    'pop',
    'reggae',
    'rock'
]

In [19]:
df['target'] = df['label'].apply(lambda x: genres_list.index(x))
df.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label,target
0,pop.00072.5.wav,66149,0.464264,0.079744,0.228116,0.007791,2156.231325,333455.342337,2295.327890,301528.295866,...,0.587634,44.336262,3.168914,31.690376,0.719836,50.416603,-3.192938,25.115688,pop,7
1,jazz.00038.9.wav,66149,0.254604,0.090115,0.075960,0.002236,1439.715643,68955.697954,1532.279764,78968.523547,...,-0.706779,105.123726,2.879812,157.346725,3.201282,71.674042,-1.664307,53.640949,jazz,5
2,disco.00055.3.wav,66149,0.488070,0.065943,0.119709,0.001558,2049.831819,132030.817465,2043.472368,71699.726714,...,-4.570374,24.995367,-0.735507,33.519482,-2.301054,38.444759,6.482006,50.393089,disco,3
3,rock.00025.9.wav,66149,0.440747,0.071972,0.108535,0.001491,2237.056815,129862.001264,2070.706749,62747.206984,...,-5.574619,29.497013,3.903063,18.930311,-2.205716,26.832264,0.551784,25.199505,rock,9
4,jazz.00013.4.wav,66149,0.369718,0.090648,0.052010,0.000124,1775.591113,66645.168399,2351.155395,51753.800714,...,-3.686353,47.710155,3.593751,70.004326,0.011506,85.980957,1.892704,108.339722,jazz,5


In [20]:
unwanted_columns = ['length','label','target']
X = df.drop(unwanted_columns+['filename'],axis='columns')
Y = df['target']


In [21]:
threshold = 0.3
df_abs = df.drop(['length','label','target','filename',],axis='columns').corr()
strong_correlations = (df_abs.abs() > threshold) & (df_abs.abs() < 1)
strong_correlation_columns = [col for col in strong_correlations.columns if any(strong_correlations[col])]
print(len(strong_correlation_columns))

56


In [22]:
X = df[strong_correlation_columns]

X = X.drop(['spectral_bandwidth_mean'],axis='columns')

In [23]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8991 entries, 0 to 8990
Data columns (total 55 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   chroma_stft_mean         8991 non-null   float64
 1   chroma_stft_var          8991 non-null   float64
 2   rms_mean                 8991 non-null   float64
 3   rms_var                  8991 non-null   float64
 4   spectral_centroid_mean   8991 non-null   float64
 5   spectral_centroid_var    8991 non-null   float64
 6   spectral_bandwidth_var   8991 non-null   float64
 7   rolloff_mean             8991 non-null   float64
 8   rolloff_var              8991 non-null   float64
 9   zero_crossing_rate_mean  8991 non-null   float64
 10  zero_crossing_rate_var   8991 non-null   float64
 11  harmony_mean             8991 non-null   float64
 12  harmony_var              8991 non-null   float64
 13  perceptr_mean            8991 non-null   float64
 14  perceptr_var            

In [24]:
Y.head()

0    7
1    5
2    3
3    9
4    5
Name: target, dtype: int64

In [25]:
def preProcessData(df, test=False):
    scaler = StandardScaler()
    if not test:
        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        return (X_train,X_test,y_train,y_test)
    else:
        scaler.fit(df)
        df = scaler.transform(df)
        return df

In [26]:
def create_model(X_train,y_train):
    mlp_classifier = MLPClassifier(hidden_layer_sizes=(512,1024,512,256,128,64),activation='tanh',solver='adam',verbose=True,max_iter=200,learning_rate_init=0.001,alpha=0.0001,random_state=42,early_stopping=True,validation_fraction=0.2,learning_rate='adaptive')
    rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42 , max_depth=25, min_samples_leaf=1, min_samples_split=2, bootstrap=False) #bootstrap is used to control the randomness of the data
    xgb_classifier = XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42,max_depth=25, subsample=0.8, colsample_bytree=0.8, gamma=0, min_child_weight=1)
    voting_classifier = VotingClassifier(estimators=[('mlp', mlp_classifier), ('rf', rf_classifier), ('xgb', xgb_classifier)],voting='hard')
    voting_classifier.fit(X_train,y_train)
    return voting_classifier

In [27]:
def checkAccuracy(model,X_test,y_test):
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

In [28]:
X_train,X_test,y_train,y_test=preProcessData(X)
model=create_model(X_train,y_train)
checkAccuracy(model,X_test,y_test)

Iteration 1, loss = 1.20238840
Validation score: 0.700486
Iteration 2, loss = 0.81379229
Validation score: 0.733843
Iteration 3, loss = 0.65921892
Validation score: 0.781098
Iteration 4, loss = 0.57012677
Validation score: 0.796386
Iteration 5, loss = 0.45708783
Validation score: 0.821404
Iteration 6, loss = 0.35488451
Validation score: 0.824878
Iteration 7, loss = 0.27689022
Validation score: 0.857540
Iteration 8, loss = 0.20223672
Validation score: 0.866574
Iteration 9, loss = 0.14129658
Validation score: 0.879778
Iteration 10, loss = 0.10339700
Validation score: 0.878388
Iteration 11, loss = 0.07341503
Validation score: 0.890202
Iteration 12, loss = 0.04691200
Validation score: 0.888812
Iteration 13, loss = 0.03498256
Validation score: 0.902015
Iteration 14, loss = 0.02290773
Validation score: 0.904795
Iteration 15, loss = 0.02169547
Validation score: 0.892981
Iteration 16, loss = 0.03899567
Validation score: 0.874913
Iteration 17, loss = 0.04249834
Validation score: 0.901320
Iterat

0.8982768204558088

In [29]:
def submission(model):
    df = pd.read_csv('./data/test.csv')
    X = df[strong_correlation_columns]
    X = X.drop(['spectral_bandwidth_mean'],axis='columns')
    IDS = df['id']

    X=preProcessData(X,test=True)
    

    y_prediction = model.predict(X)

    y_prediction_series = pd.Series(y_prediction,name='label')

    df_ans = pd.concat([IDS, y_prediction_series], axis=1)
    df_ans.to_csv('output_0.csv', index=False)
    print(df_ans.head())

submission(model)

     id  label
0  9586      9
1  6984      4
2  7047      7
3  2510      2
4    34      0


In [30]:
def analyseAns():
    ans = pd.read_csv('./data/baseline.csv')
    res = pd.read_csv('./output_0.csv')
    y_pred = ans['label']
    y_ans = res['label']

    print(accuracy_score(y_ans,y_pred))

analyseAns()

0.9179179179179179
